In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# library imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [34]:
# scikit imports
from sklearn.preprocessing import Imputer

In [4]:
bank_data_set = pd.read_csv("dataset/bank-additional.csv", sep=";");

In [6]:
bank_data_set.shape

(4119, 21)

In [7]:
bank_data_set.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [8]:
bank_data_set.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [9]:
bank_data_set.describe(include='all')

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,4119.000000,4119,4119,4119,4119,4119,4119,4119,4119,4119,...,4119.000000,4119.000000,4119.000000,4119,4119.000000,4119.000000,4119.000000,4119.000000,4119.000000,4119
unique,NaN,12,4,8,3,3,3,2,10,5,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,thu,...,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,1012,2509,1264,3315,2175,3349,2652,1378,860,...,NaN,NaN,NaN,3523,NaN,NaN,NaN,NaN,NaN,3668
mean,40.113620,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.537266,960.422190,0.190337,NaN,0.084972,93.579704,-40.499102,3.621356,5166.481695,NaN
std,10.313362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.568159,191.922786,0.541788,NaN,1.563114,0.579349,4.594578,1.733591,73.667904,NaN
min,18.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.635000,4963.600000,NaN
25%,32.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.334000,5099.100000,NaN
50%,38.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,999.000000,0.000000,NaN,1.100000,93.749000,-41.800000,4.857000,5191.000000,NaN
75%,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000,NaN


In [24]:
# where pdays is 999: client was not previously contacted.
# more than 95% was not contacted, we should probably drop this feature or turn it into binary.
(bank_data_set["pdays"] == 999).sum() / len(bank_data_set)

0.9611556202961884

In [27]:
(bank_data_set["previous"] == 0).sum() / len(bank_data_set)

0.85530468560330175

In [26]:
(bank_data_set["poutcome"] == "nonexistent").sum() / len(bank_data_set)

0.85530468560330175

# Cleaning the dataset 

In [32]:
nominal_features = ["job", "marital", "education", "default", "housing", 
                    "loan", "contact", "month", "day_of_week", "poutcome",
                   ]
numeric_features = ["age", "campaign", "pdays", "previous", "emp.var.rate", 
                    "cons.price.idx", "euribor3m", "nr.employed",
                   ]

In [33]:
for feature in nominal_features:
    nr_missing = (bank_data_set[feature] == "unknown").sum()
    ratio = nr_missing / len(bank_data_set)
    print (feature, nr_missing, ratio)

job 39 0.0094683175528
marital 11 0.00267055110464
education 167 0.0405438213159
default 803 0.194950230639
housing 105 0.0254916241806
loan 105 0.0254916241806
contact 0 0.0
month 0 0.0
day_of_week 0 0.0
poutcome 0 0.0


In [85]:
bank_df = (bank_data_set[(bank_data_set["job"] != "unknown") & 
                        (bank_data_set["marital"] != "unknown") &
                        (bank_data_set["education"] != "unknown") &
                        (bank_data_set["housing"] != "unknown") &
                        (bank_data_set["loan"] != "unknown")
                       ]).copy()
bank_df.reset_index(inplace=True, drop=True)

In [96]:
# impute values of default
mode = bank_df["default"].mode()
actual_mode = mode[0]
bank_df["default"] = (bank_df["default"]).replace("unknown", actual_mode)

In [97]:
bank_df.describe(include="all")

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
count,3811.000000,3811,3811,3811,3811,3811,3811,3811,3811,3811,...,3811.000000,3811.000000,3811.000000,3811,3811.000000,3811.000000,3811.000000,3811.000000,3811.000000,3811
unique,NaN,11,3,7,2,2,2,2,10,5,...,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,2
top,NaN,admin.,married,university.degree,no,yes,no,cellular,may,mon,...,NaN,NaN,NaN,nonexistent,NaN,NaN,NaN,NaN,NaN,no
freq,NaN,970,2330,1239,3810,2072,3177,2470,1275,801,...,NaN,NaN,NaN,3268,NaN,NaN,NaN,NaN,NaN,3398
mean,39.961952,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.539491,961.467331,0.184204,NaN,0.088113,93.575567,-40.560798,3.626035,5167.004749,NaN
std,10.215878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.590149,189.427840,0.523136,NaN,1.557160,0.578653,4.602293,1.727428,73.160799,NaN
min,19.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,0.000000,0.000000,NaN,-3.400000,92.201000,-50.800000,0.635000,4963.600000,NaN
25%,32.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,999.000000,0.000000,NaN,-1.800000,93.075000,-42.700000,1.334000,5099.100000,NaN
50%,38.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.000000,999.000000,0.000000,NaN,1.100000,93.444000,-41.800000,4.857000,5191.000000,NaN
75%,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.000000,999.000000,0.000000,NaN,1.400000,93.994000,-36.400000,4.961000,5228.100000,NaN
